In [1]:
import gempipe

from helper_functions import *

In [2]:
tools = ['gempipe', 'gempipe_rf', 'carveme', 'bactabolize', 'gapseq']

# Main functions

In [3]:
def match_biolog_data(dataset, tools):
    
    # mach biolog sims with experimental data
    
    
    # import experimental data:
    df_match = pnd.read_csv(f'{dataset}/tables/biolog_exp.csv', index_col=0)
    biolog_mappings = get_biolog_C_mappings()
    

    for tool in tools: 
        df_match[f'model_value_{tool}'] = None
        df_match[f'call_{tool}'] = None


    # iterate through each strain:
    for tool, biolog_sims in zip(tools, [
        glob.glob(f'{dataset}/tools_runs/gempipe/output/biolog/*.csv'), 
        glob.glob(f'{dataset}/tools_runs/gempipe_rf/output/biolog/*.csv'), 
        glob.glob(f'{dataset}/tools_runs/carveme/biolog_sims/*.csv'), 
        glob.glob(f'{dataset}/tools_runs/bactabolize/biolog_sims/*.csv'),
        glob.glob(f'{dataset}/tools_runs/gapseq/biolog_sims/*.csv'),
    ]):

        for ss_biolog in biolog_sims:
            # load the simulations
            strain_id = Path(ss_biolog).stem 
            strain_id = strain_id.replace('_model', '')
            ss_biolog = pnd.read_csv(ss_biolog, index_col=0)


            # fill the table:
            cnt = 0
            for index, row in df_match.iterrows():
                if row['strain'] == strain_id:
                    substrate = row['substrate']


                    # extract the corresponding row from simulations:
                    match = ss_biolog[(ss_biolog['substrate']==substrate) & (ss_biolog['source']=='C')].iloc[0]


                    model_value_tool = None

                    if substrate in biolog_mappings[biolog_mappings['BiGG_exchange'].notna()].index:
                        model_value_tool = 0
                    if match['growth']==True:  model_value_tool = 1 
                    if match['growth']==False: model_value_tool = 0 

                    df_match.loc[index, f'model_value_{tool}'] = model_value_tool 


                    # track TP / TN / FP / FN looking at 'growth_call':
                    if model_value_tool != None:
                        if model_value_tool == row['growth_call'] and row['growth_call'] == 1:
                            df_match.loc[index, f'call_{tool}'] = 'TP' 
                            cnt += 1

                        elif model_value_tool == row['growth_call'] and row['growth_call'] == 0:
                            df_match.loc[index, f'call_{tool}'] = 'TN' 
                            cnt += 1

                        elif model_value_tool != row['growth_call'] and row['growth_call'] == 0:
                            df_match.loc[index, f'call_{tool}'] = 'FP' 
                            cnt += 1

                        elif model_value_tool != row['growth_call'] and row['growth_call'] == 1:
                            df_match.loc[index, f'call_{tool}'] = 'FN' 
                            cnt += 1


                    if match['status']=='infeasible':
                        df_match.loc[index, f'call_{tool}'] = 'infeasible' 

            print(tool, '\t', strain_id, '\t', cnt)
 
    return df_match

In [4]:

def get_FN_counts(dataset, df_match):
    # get FN counts; consider also the pan-model's Biolog sims
    
    # load pan-model's Biolog sims:
    biolog_panmodel = pnd.read_csv(f'{dataset}/tools_runs/gempipe/output/biolog_panmodel.csv', index_col=0)
    biolog_mappings = get_biolog_C_mappings()


    # try to understand FN mismatches
    fns_all = df_match.copy()
    fns_all = fns_all[(fns_all['growth_call']==1) & (fns_all['call_gempipe']=='FN')]
    fns_all = fns_all[['substrate', 'growth_call', 'call_gempipe']]
    fns = fns_all.drop_duplicates().copy()


    # count the number of occurences
    fns['counter'] = None
    for index, row in fns.iterrows():
        group = fns_all[fns_all['substrate']==row['substrate']]
        fns.loc[index, 'counter'] = len(group)
    fns = fns.sort_values('counter', ascending=False)


    # add the corresponding EX reaction: 
    fns['exr'] = None
    for index, row in fns.iterrows():
        substrate = row['substrate']
        fns.loc[index, 'exr'] = biolog_mappings.loc[substrate, 'BiGG_exchange'] 


    # test if the panmodel can grow or not
    fns['pan_growth'] = None
    for index, row in fns.iterrows(): 
        substrate = row['substrate']
        match = biolog_panmodel[(biolog_panmodel['substrate']==substrate) & (biolog_panmodel['source']=='C')].iloc[0]
        fns.loc[index, f'pan_growth'] = match['value']

    fns = fns.reset_index(drop=True)
    fns = fns.set_index('substrate', drop=True)
    
    return fns

# 01_klebsiella

In [5]:
df_match = match_biolog_data(dataset='01_klebsiella', tools=tools)
df_match.to_csv('01_klebsiella/tables/biolog_match.csv')
df_match

gempipe 	 Kvt_CDC4241_71 	 90
gempipe 	 Kp_KP13 	 90
gempipe 	 Kp_04A025 	 90
gempipe 	 Kqs_07A044T 	 90
gempipe 	 Ka_200023T 	 90
gempipe 	 Kp_BJ1 	 90
gempipe 	 Kqq_SB98 	 90
gempipe 	 Kp_SB1067 	 90
gempipe 	 Kvv_F2R9T 	 90
gempipe 	 Kp_SA12 	 90
gempipe 	 Kp_SB1139 	 90
gempipe 	 Kp_MGH78578 	 90
gempipe 	 Kvv_342 	 90
gempipe 	 Kqs_CIP110288 	 90
gempipe 	 Kp_NJST258_1 	 90
gempipe 	 Kqq_18A069 	 90
gempipe 	 Kp_SB611 	 90
gempipe 	 Kp_T69 	 90
gempipe 	 Kp_SB617 	 90
gempipe 	 Kvv_01A065 	 90
gempipe 	 Kp_SB1170 	 89
gempipe 	 Kp_SB615 	 90
gempipe 	 Kp_SB2390 	 90
gempipe 	 Kp_SB612 	 90
gempipe 	 Kp_NTUHK2044 	 90
gempipe 	 Kp_CIP52.145 	 90
gempipe 	 Kqv_08A119 	 90
gempipe 	 Kqs_09A323 	 90
gempipe 	 Kp_CG43 	 90
gempipe 	 Kp_SA1 	 90
gempipe 	 Kqs_12A476 	 90
gempipe 	 Kqq_U41 	 90
gempipe 	 Kqq_SB1124 	 90
gempipe 	 Kqq_01A030T 	 90
gempipe 	 Kp_03_9138_2 	 90
gempipe 	 Kvv_At_22 	 90
gempipe 	 Kqs_SB610 	 90
gempipe_rf 	 Kvt_CDC4241_71 	 90
gempipe_rf 	 Kp_KP13 	 90
gempip

,strain,substrate,growth_call,model_value_gempipe,call_gempipe,model_value_gempipe_rf,call_gempipe_rf,model_value_carveme,call_carveme,model_value_bactabolize,call_bactabolize,model_value_gapseq,call_gapseq
2,Kvv_01A065,g-Amino-N-Butyric acid,1.0,1,TP,1,TP,1,TP,1,TP,1,TP
3,Kvv_01A065,L-Sorbose,1.0,0,FN,0,FN,0,FN,0,FN,0,FN
4,Kvv_01A065,Mucic acid,1.0,1,TP,1,TP,1,TP,1,TP,0,FN
5,Kvv_01A065,Tricarballylic acid,0.0,0,TN,0,TN,1,FP,0,TN,0,TN
8,Kp_SB1067,g-Amino-N-Butyric acid,1.0,1,TP,1,TP,1,TP,1,TP,1,TP
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3473,Kp_SB612,Dihydroxyacetone,1.0,1,TP,0,FN,1,TP,1,TP,0,FN
3474,Kp_SB615,Dihydroxyacetone,1.0,1,TP,0,FN,1,TP,1,TP,0,FN
3475,Kp_SB617,Dihydroxyacetone,1.0,1,TP,0,FN,1,TP,1,TP,0,FN
3476,Kvt_CDC4241_71,Dihydroxyacetone,1.0,1,TP,0,FN,1,TP,1,TP,0,FN


In [6]:
fns = get_FN_counts('01_klebsiella', df_match)
fns.to_csv('01_klebsiella/tables/biolog_fns.csv')
fns

,growth_call,call_gempipe,counter,exr,pan_growth
substrate,,,,,
Ala-Gly,1.0,FN,37,EX_L_alagly_e,NaN
Arbutin,1.0,FN,37,EX_arbt_e,0.0
Palatinose,1.0,FN,37,EX_pala_e,NaN
D-Raffinose,1.0,FN,37,EX_raffin_e,NaN
Turanose,1.0,FN,37,EX_tur_e,NaN
Tyramine,1.0,FN,36,EX_tym_e,0.0
Gly-Pro,1.0,FN,35,EX_gly_pro__L_e,0.0
L-Sorbose,1.0,FN,18,EX_srb__L_e,NaN
Tricarballylic acid,1.0,FN,15,EX_tcb_e,NaN


# 02_ralstonia

In [7]:
df_match = match_biolog_data(dataset='02_ralstonia', tools=tools)
df_match.to_csv('02_ralstonia/tables/biolog_match.csv')
df_match

gempipe 	 K60 	 147
gempipe 	 PSI07 	 147
gempipe 	 PSS4 	 147
gempipe 	 GMI1000 	 147
gempipe 	 MOLK2 	 147
gempipe 	 UW551 	 147
gempipe 	 BDBR229 	 147
gempipe 	 RUN2340 	 147
gempipe 	 R24 	 147
gempipe 	 BA7 	 147
gempipe 	 CFBP2957 	 147
gempipe_rf 	 K60 	 147
gempipe_rf 	 PSI07 	 147
gempipe_rf 	 PSS4 	 147
gempipe_rf 	 GMI1000 	 147
gempipe_rf 	 MOLK2 	 147
gempipe_rf 	 UW551 	 147
gempipe_rf 	 BDBR229 	 147
gempipe_rf 	 RUN2340 	 147
gempipe_rf 	 R24 	 147
gempipe_rf 	 BA7 	 147
gempipe_rf 	 CFBP2957 	 147
carveme 	 K60 	 147
carveme 	 PSI07 	 147
carveme 	 PSS4 	 147
carveme 	 GMI1000 	 147
carveme 	 MOLK2 	 147
carveme 	 UW551 	 147
carveme 	 BDBR229 	 147
carveme 	 RUN2340 	 147
carveme 	 R24 	 147
carveme 	 BA7 	 147
carveme 	 CFBP2957 	 147
bactabolize 	 UW551 	 147
bactabolize 	 CFBP2957 	 147
bactabolize 	 BDBR229 	 147
bactabolize 	 GMI1000 	 147
bactabolize 	 RUN2340 	 147
bactabolize 	 PSI07 	 147
bactabolize 	 R24 	 147
bactabolize 	 K60 	 147
bactabolize 	 BA7 	 14

,strain,substrate,growth_call,model_value_gempipe,call_gempipe,model_value_gempipe_rf,call_gempipe_rf,model_value_carveme,call_carveme,model_value_bactabolize,call_bactabolize,model_value_gapseq,call_gapseq
0,BA7,L-Arabinose,0,0,TN,0,TN,0,TN,0,TN,1,FP
1,PSS4,L-Arabinose,0,0,TN,0,TN,0,TN,0,TN,1,FP
2,GMI1000,L-Arabinose,0,0,TN,0,TN,0,TN,0,TN,1,FP
3,R24,L-Arabinose,0,0,TN,0,TN,0,TN,0,TN,1,FP
4,PSI07,L-Arabinose,0,0,TN,0,TN,0,TN,0,TN,1,FP
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2085,UW551,3-hydroxy-2-butanone,0,0,TN,0,TN,0,TN,0,TN,0,TN
2086,MOLK2,3-hydroxy-2-butanone,0,0,TN,0,TN,0,TN,0,TN,0,TN
2087,RUN2340,3-hydroxy-2-butanone,0,0,TN,0,TN,0,TN,0,TN,0,TN
2088,K60,3-hydroxy-2-butanone,0,0,TN,0,TN,0,TN,0,TN,0,TN


In [8]:
fns = get_FN_counts('02_ralstonia', df_match)
fns.to_csv('02_ralstonia/tables/biolog_fns.csv')
fns

,growth_call,call_gempipe,counter,exr,pan_growth
substrate,,,,,
quinic acid,1,FN,10,EX_quin_e,NaN
pectin,1,FN,8,EX_pectin_e,NaN
Dihydroxyacetone,1,FN,5,EX_dha_e,1.698058
N-Acetyl-D-Glucosamine,1,FN,3,EX_acgam_e,NaN
p-Hydroxyphenyl Acetic acid,1,FN,3,EX_4hphac_e,20.936671
L-Galactonic Acid-gamma-Lactone,1,FN,3,EX_galctn__L_e,NaN
d-tartaric acid,1,FN,2,EX_tartr__D_e,NaN
sebacic acid,1,FN,2,EX_sebacid_e,NaN
D-Aspartic Acid,1,FN,2,EX_asp__D_e,NaN


# 03_pseudomonas

In [9]:
df_match = match_biolog_data(dataset='03_pseudomonas', tools=tools)
df_match.to_csv('03_pseudomonas/tables/biolog_match.csv')
df_match

gempipe 	 ChPhzTR39 	 93
gempipe 	 Q16 	 93
gempipe 	 ChPhzTR18 	 93
gempipe 	 PCM2210 	 93
gempipe 	 DSM_19603 	 93
gempipe 	 66 	 93
gempipe 	 SLPH10 	 93
gempipe 	 CW2 	 93
gempipe 	 ATCC17411 	 93
gempipe 	 449 	 93
gempipe 	 K27 	 93
gempipe 	 PA23 	 93
gempipe 	 P2 	 93
gempipe 	 DSM_21509 	 93
gempipe 	 ChPhzS140 	 93
gempipe 	 M71 	 93
gempipe 	 DTR133 	 93
gempipe 	 O6 	 93
gempipe 	 PCL1607 	 93
gempipe 	 ChPhzS24 	 93
gempipe 	 ATCC17809 	 93
gempipe 	 ChPhzS135 	 93
gempipe 	 30-84 	 93
gempipe 	 ToZa7 	 93
gempipe 	 ATCC17415 	 93
gempipe 	 ChPhzS23 	 93
gempipe 	 464 	 93
gempipe 	 PCL1391 	 93
gempipe 	 ChPhzTR38 	 93
gempipe 	 ChPhzTR44 	 93
gempipe 	 TAMOak81 	 93
gempipe 	 C50 	 93
gempipe 	 DSM_6698 	 93
gempipe 	 B25 	 93
gempipe 	 Pb-St2 	 93
gempipe 	 M12 	 93
gempipe_rf 	 ChPhzTR39 	 93
gempipe_rf 	 Q16 	 93
gempipe_rf 	 ChPhzTR18 	 93
gempipe_rf 	 PCM2210 	 93
gempipe_rf 	 DSM_19603 	 93
gempipe_rf 	 66 	 93
gempipe_rf 	 SLPH10 	 93
gempipe_rf 	 CW2 	 93
gempipe

,strain,substrate,growth_call,model_value_gempipe,call_gempipe,model_value_gempipe_rf,call_gempipe_rf,model_value_carveme,call_carveme,model_value_bactabolize,call_bactabolize,model_value_gapseq,call_gapseq
0,30-84,L-Arabinose,1,0,FN,0,FN,0,FN,0,FN,1,TP
1,ATCC17415,L-Arabinose,1,0,FN,0,FN,0,FN,0,FN,1,TP
2,TAMOak81,L-Arabinose,0,0,TN,0,TN,0,TN,0,TN,1,FP
3,ATCC17411,L-Arabinose,0,0,TN,0,TN,0,TN,0,TN,1,FP
4,ATCC17809,L-Arabinose,0,0,TN,0,TN,0,TN,0,TN,1,FP
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3955,ChPhzTR38,N-Acetyl-D-Mannosamine,1,0,FN,0,FN,0,FN,0,FN,0,FN
3956,ChPhzTR39,N-Acetyl-D-Mannosamine,1,0,FN,0,FN,0,FN,0,FN,0,FN
3957,ChPhzTR18,N-Acetyl-D-Mannosamine,1,0,FN,0,FN,0,FN,0,FN,0,FN
3958,PA23,N-Acetyl-D-Mannosamine,1,0,FN,0,FN,0,FN,0,FN,0,FN


In [10]:
fns = get_FN_counts('03_pseudomonas', df_match)
fns.to_csv('03_pseudomonas/tables/biolog_fns.csv')
fns

,growth_call,call_gempipe,counter,exr,pan_growth
substrate,,,,,
Ala-Gly,1,FN,36,EX_L_alagly_e,NaN
D-Mannitol,1,FN,36,EX_mnl_e,NaN
Tyramine,1,FN,35,EX_tym_e,NaN
n-acetyl-l-glutamic acid,1,FN,35,EX_acglu_e,NaN
N-Acetyl-D-Glucosamine,1,FN,35,EX_acgam_e,NaN
m-Inositol,1,FN,34,EX_inost_e,0.0
p-Hydroxyphenyl Acetic acid,1,FN,34,EX_4hphac_e,21.137468
malonic acid,1,FN,33,EX_malon_e,NaN
N-Acetyl-D-Mannosamine,1,FN,33,EX_acmana_e,0.0


# Create supplementary table

In [40]:
res_dic = {}
for dataset in [
    '01_klebsiella',
    '02_ralstonia',
    '03_pseudomonas',
]:
    df_match = pnd.read_csv(f'{dataset}/tables/biolog_match.csv', index_col=0)
    
    # remove substrates never modeled by any of the tools
    for tool in ['gempipe', 'gempipe_rf', 'carveme', 'bactabolize', 'gapseq']:
        df_match = df_match[
            df_match[f'model_value_{tool}'].notna()
        ]
    
    res_dic[dataset] = {}
    for substrate in df_match['substrate'].unique():
        res_dic[dataset][substrate] = {} 
    
    for index, row in df_match.iterrows():
        
        try: values = [str(int(row['growth_call']))]
        except: values = ['?']
        
        for tool in ['gempipe', 'gempipe_rf', 'carveme', 'bactabolize', 'gapseq']:
            call = str(df_match.loc[index, f'call_{tool}'])
            if call=='infeasible': call = 'in' # shorter
            values = values + [call]
        
        res_dic[dataset][row['substrate']][row['strain']] = ','.join(values)

        
with pnd.ExcelWriter('tables/Supplementary File 1.xlsx', engine='xlsxwriter') as writer:
    
    
    INFO = """
        Comparison between experimental and simulated binarized Biolog® PM screenings. 
        Rows are substrates, columns are strains. 
        Cells follow the format “{E}-{T1}-{T2}-{T3}-{T4}-{T5}”, 
        where {E} is the experimental binary growth output (0: no growth; 1: growth); 
        {T1}, {T2}, {T3}, {T4}, {T5} are the matches of Gempipe, Gempipe (reference-free), CarveMe, Bactabolize and gapse, respectively.
        (“TP”: true positive; “TN”: true negative; “FP”: false positive; “FN”: false negative; “in”: infeasible FBA solution).
    """
    data = [i.strip().rstrip() for i in INFO.strip().rstrip().split('\n')]
    df = pnd.DataFrame(data, columns=["INFO"])
    df.to_excel(writer, sheet_name='INFO', index=False)
    
    for dataset in res_dic.keys():
        
        rows = []  # list of dicts
        for substrate in res_dic[dataset].keys():
            row = {}
            row['substrate'] = substrate
            for strain in res_dic[dataset][substrate].keys():
                row[strain] = res_dic[dataset][substrate][strain]
            rows.append(row)
                
        df =  pnd.DataFrame.from_records(rows)     
        df.to_excel(writer, sheet_name=dataset, index=False)
        
